1 Router Agent

Router Agent 是 Multi-Agent 系统的核心部分，它负责分析问题，并决定将任务分发到哪个Agent上。我们利用 Assistant API 创建 Router Agent，并通过 instructions 指定其可调用的 Agent 及输出格式。

由于 Agent 的返回值为字符串格式，我们要求 Router Agent 输出列表形式的字符串，例如：["employee_info_agent", "leave_agent", "company_info_agent"]，以便后续使用字符串解析工具将其转换为结构化数据。

In [1]:
# 导入依赖
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage

# 定义一个员工查询函数
def query_employee_info(query):
    # 初始化 DashScope LLM
    llm = DashScope(model="qwen-plus")

    # 创建消息列表，包含系统提示和用户查询
    messages = [
        ChatMessage(role=MessageRole.SYSTEM, content='''你有一个表叫employees，记录公司的员工信息，这个表有department（部门）、name（姓名）、HR三个字段。
    你需要根据用户输入生成sql语句进行查询,你一定不能生成sql语句之外的内容，也不要把```sql```这个信息加上。'''),
        ChatMessage(role=MessageRole.USER, content=query)
    ]

    # 发送消息到 LLM 并获取响应
    SQL_output = llm.chat(messages).message.content

    print(f"生成的SQL语句是: {SQL_output}")
    # 2. 根据SQL语句去查询数据库（此处为模拟查询），并返回结果
    if SQL_output == "SELECT COUNT(*) FROM employees WHERE department = '教育部门'":
        return "教育部门共有66名员工。"
    if SQL_output == "SELECT HR FROM employees WHERE name = '张三'":
        return "张三的HR是李四。"
    if SQL_output == "SELECT department FROM employees WHERE name = '王五'":
        return "王五的部门是后勤部。"
    else:
        return "抱歉，我暂时无法回答您的问题。"

def send_leave_application(date):
  '''
  输入请假时间，输出请假申请发送结果
  '''
  return f'已为您提交{date}的请假申请。'

import sys
sys.path.append('..')
from chatbot.rag import load_index

def query_company_info(query):
    index = load_index()
    query_engine = index.as_query_engine(
      llm=DashScope(model="qwen-plus")
    )
    return query_engine.query(query).response

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/miniconda3/envs/ACPEnv/lib/python3.13/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
from dashscope import Assistants, Messages, Runs, Threads
import json

# 决策级别的agent，决定使用哪些agent，以及它们的运行顺序
router_agent = Assistants.create(
  model="qwen-plus",
  # 定义Agent的名称
  name="流程编排机器人",
  # 定义Agent的功能描述
  description="你是团队的leader，你的手下有很多agent，你需要根据用户的输入，决定要以怎样的顺序去使用这些agent",
  # 定义对Agent的指示语句，Agent会按照指示语句进行工具的调用并返回结果。
  instructions="""你的团队中有以下agent。
    employee_info_agent：可以查询公司的员工信息，如果提问中关于部门、HR等信息，则调用该agent；
    leave_agent：可以帮助员工发送请假申请，如果用户提出请假，则调用该agent；
    company_info_agent: 可以查询公司的规章制度信息，如果用户提出关于公司规章制度的问题，则调用该agent；
    chat_agent：如果用户的问题无需以上agent，则调用该agent。

    你需要根据用户的问题，判断要以什么顺序使用这些agent，一个agent可以被多次调用。你的返回形式是一个列表，不能返回其它信息。
    比如：["employee_info_agent", "leave_agent","company_info_agent"]或者["chat_agent"]，列表中的元素只能为上述的agent。
"""
)

print("Router Agent 创建完成")
# 建立Agent Function name与工具函数的映射关系
function_mapper = {
    "查询员工信息": query_employee_info,
    "发送请假申请": send_leave_application,
    "查询公司规章制度": query_company_info
}

print("Function Mapper 创建完成")

# 输入message信息，输出为指定Agent的回复
def get_agent_response(assistant, message=''):
    # 打印出输入Agent的信息
    thread = Threads.create()
    message = Messages.create(thread.id, content=message)
    run = Runs.create(thread.id, assistant_id=assistant.id)
    run_status = Runs.wait(run.id, thread_id=thread.id)
    # 如果响应失败，会打印出run failed
    if run_status.status == "failed":
        print("Run failed：")
    # 如果需要工具来辅助大模型输出，则进行以下流程
    if run_status.required_action:
      f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
      # 获得function name
      func_name = f['name']
      # 获得function 的入参
      param = json.loads(f['arguments'])
      # 打印出工具信息
      print(f"需要调用工具: {func_name}，参数是: {param}", f)
      # 根据function name，通过function_mapper映射到函数，并将参数输入工具函数得到output输出
      if func_name in function_mapper:
          output = function_mapper[func_name](**param)
      else:
          output = ""
      tool_outputs = [{
        'output': output,}]
      # 将工具函数的输出，传入到agent中，继续完成agent的任务
      run = Runs.submit_tool_outputs(
          run.id,
          thread_id=thread.id,
          tool_outputs=tool_outputs
      )
      run_status = Runs.wait(run.id, thread_id=thread.id)

    run_status = Runs.get(run.id, thread_id=thread.id)
    msgs = Messages.list(thread.id)
    # 打印出agent的回复
    return msgs['data'][0]['content'][0]['text']['value']

Router Agent 创建完成
Function Mapper 创建完成


我们准备了几个测试问题，看下Router Agent能否分发到正确的Agent。

In [12]:
query_stk = [
    "谁是张三的HR？教育部门一共有多少员工？",
    "王五在哪个部门？帮我提交下周三请假的申请",
    "你好"
]
for query in query_stk:
    print(f"用户的问题是: {query}")
    agent_response = get_agent_response(router_agent, query)
    print(f"Agent的回复是: {agent_response}")

用户的问题是: 谁是张三的HR？教育部门一共有多少员工？
Agent的回复是: ["employee_info_agent", "employee_info_agent"]
用户的问题是: 王五在哪个部门？帮我提交下周三请假的申请
Agent的回复是: ["employee_info_agent", "leave_agent"]
用户的问题是: 你好
Agent的回复是: ["chat_agent"]


对于这三个测试问题，Router Agent都做出了正确的选择。

Router Agent在给出了回复后，你需要将它解析为列表数据以便在后续进行处理，此处使用ast工具加载列表形式的字符串。

In [14]:
import ast
# 获得 Router Agent 的回复
router_response = get_agent_response(router_agent, "谁是张三的HR？教育部门一共有多少员工？帮我提交下周三请假的申请")
# 通过ast工具将列表形式的字符串加载为列表
order_stk = ast.literal_eval(router_response)
# 打印出Router Agent的规划
for i in range(len(order_stk)):
    print(f"第{i+1}步调用的Agent是: {order_stk[i]}")


第1步调用的Agent是: employee_info_agent
第2步调用的Agent是: employee_info_agent
第3步调用的Agent是: leave_agent


2 执行工具函数的Agent

在获取到Router Agent的规划后，我们需要进行执行工具函数Agent的创建。创建方法与第二节基本一致，区别在于将三个工具函数分配到三个Agent上。

需要确保agent变量名与Router Agent的instructions中定义的agent变量名一致。

In [ ]:
# 员工信息查询agent
employee_info_agent = Assistants.create(
  model="qwen-plus",
  name="员工信息查询机器人",
  description="你是一个员工信息查询机器人，你可以根据用户的问题，查询公司的员工信息",
  instructions="你是员工信息查询助手，负责查询员工姓名、部门、HR等信息。",
  tools=[
    {
      'type': 'function',
      'function': {
        'name': '查询员工信息',
        'description': '当需要查询员工信息时非常有用，比如查询员工张三的HR是谁，查询教育部门总人数等。',
        'parameters': {
          'type': 'object',
          'properties': {
            'query': {
              'type': 'str',
              'description': '用户的问题'
            }
          },
          'required': ['query']
        }
      }
    }
  ]
)
print(f"员工信息查询agent {employee_info_agent.name} 创建完成")

# 请假申请agent
leave_agent = Assistants.create(
  model="qwen-plus",
  name="请假申请机器人",
  description="你是一个请假申请机器人，你可以根据用户的请求，帮助用户发送请假申请",
  instructions="你是请假申请助手，负责帮助员工发送请假申请。",
  tools=[
    {
      'type': 'function',
      'function': {
        'name': '发送请假申请',
        'description': '当用户需要请假时非常有用，比如用户说我要请假，帮我提交请假申请等。',
        'parameters': {
          'type': 'object',
          'properties': {
            # 需要请假的时间
            'date': {
              'type': 'str',
              'description': '请假的时间'
            }
          },
          'required': ['date']
        }
      }
    }
  ]
)
print(f"请假申请agent {leave_agent.name} 创建完成")

# 公司规章制度查询agent
company_info_agent = Assistants.create(
  model="qwen-plus",
  name="公司规章制度查询机器人",
  description="你是一个公司规章制度查询机器人，你可以根据用户的问题，查询公司的规章制度信息",
  instructions="你是公司规章制度查询助手，负责查询公司的规章制度信息。",
  tools=[
    {
      'type': 'function',
      'function': {
        'name': '查询公司规章制度',
        'description': '当需要查询公司的规章制度时非常有用，比如公司的考勤制度是什么，公司有哪些福利等。',
        'parameters': {
          'type': 'object',
          'properties': {
            'query': {
              'type': 'str',
              'description': '用户的问题'
            }
          },
          'required': ['query']
        }
      }
    }
  ]
)
print(f"公司规章制度查询agent {company_info_agent.name} 创建完成")

# 功能是回复日常问题。对于日常问题来说，可以使用价格较为低廉的模型作为agent的基座
chat_agent = Assistants.create(
  model="qwen-plus",
  name="日常问题回复机器人",
  description="你是一个日常问题回复机器人，你可以根据用户的问题，回复用户的日常问题",
  instructions="你是日常问题回复助手，负责回复用户的日常问题。",
)
print(f"日常问题回复agent {chat_agent.name} 创建完成")


员工信息查询agent 员工信息查询机器人 创建完成
请假申请agent 请假申请机器人 创建完成
公司规章制度查询agent 公司规章制度查询机器人 创建完成
日常问题回复agent 日常问题回复机器人 创建完成


3 创建Summary Agent并测试Multi-Agent效果

在完成了Router Agent和执行工具函数Agent的创建后，我们还需要创建Summary Agent，该Agent会根据用户的问题与之前Agent输出的参考信息，全面、完整地回答用户问题.

In [16]:
summary_agent = Assistants.create(
  model="qwen-plus",
  name="总结机器人",
  description="你是一个总结机器人，你可以根据用户的问题，给出一个简短的总结",
  instructions="你是总结助手，负责给出一个简短的总结。",
)
print(f"总结agent {summary_agent.name} 创建完成")

总结agent 总结机器人 创建完成


我们将以上步骤封装为一个get_multi_agent_response函数，这样只需要输入问题就可以得到回复了。

由于列表中的元素为字符串，因此通过一个agent_mapper方法将字符串格式的Agent映射到定义好的Agent对象。

In [40]:
# 将列表中的字符串映射到Agent对象上
agent_mapper = {
  "employee_info_agent": employee_info_agent,
  "leave_agent": leave_agent,
  "company_info_agent": company_info_agent,
  # "chat_agent": chat_agent,
}

def get_multi_agent_response(query):
  # 1. 让Router Agent进行规划，决定调用哪些Agent
  # 获取Agent的运行顺序
  agent_order = get_agent_response(router_agent, query)

  # 2. 按照Router Agent的规划，依次调用各个Agent
  # 由于大模型输出可能不稳定，因此加入异常处理模块处理列表字符串解析失败的问题

  try:

    order_stk = ast.literal_eval(agent_order)
    print(f"Router Agent的规划是: {order_stk}")
    print("Router Agent正在工作：")
    for i in range(len(order_stk)):
      print(f"第{i+1}步调用的Agent是: {order_stk[i]}")
    # 随着多Agent的加入，需要将Agent的输出添加到用户问题中，作为参考信息
    cur_query = query
    Agent_Message = ""

    # 依次运行Agent
    for i in range(len(order_stk)):
      cur_agent = agent_mapper[order_stk[i]]
      response = get_agent_response(cur_agent, cur_query)
      Agent_Message += f"第{i+1}步调用的{order_stk[i]}的回复是: {response}\n"
      # 如果当前Agent为最后一个Agent，则将其输出作为Multi Agent的输出
      if i == len(order_stk) - 1:
        prompt = f"请参考已知的信息：{Agent_Message}，回答用户的问题：{query}。"
        multi_agent_response = get_agent_response(summary_agent, prompt)
        print(f"Multi Agent的回复是: {multi_agent_response}")
        return multi_agent_response
      # 如果当前Agent不是最后一个Agent，则将上一个Agent的输出response添加到下一轮的query中，作为参考信息
      else:
          # 如果不是最后一个Agent，则将当前Agent的输出添加到用户问题中，作为下一个Agent的输入
          # cur_query += f"。中间结果：{response}"
          cur_query = f"你可以参考已知的信息：{response}你要完整地回答用户的问题。问题是：{query}。"
  except Exception as e:
    print(f"解析Router Agent的输出时出错: {e}")
    return get_agent_response(chat_agent, query)

# 测试Multi Agent
get_multi_agent_response("王五在哪个部门？帮我提交下周三请假的申请")

Router Agent的规划是: ['employee_info_agent', 'leave_agent']
Router Agent正在工作：
第1步调用的Agent是: employee_info_agent
第2步调用的Agent是: leave_agent
需要调用工具: 查询员工信息，参数是: {'query': '王五在哪个部门'} {'name': '查询员工信息', 'arguments': '{"query": "王五在哪个部门"}', 'output': None}
生成的SQL语句是: SELECT department FROM employees WHERE name = '王五'
需要调用工具: 发送请假申请，参数是: {'date': '2025年9月24日'} {'name': '发送请假申请', 'arguments': '{"date": "2025年9月24日"}', 'output': None}
Multi Agent的回复是: 王五在后勤部。已为您提交2025年9月24日（下周三）的请假申请。


'王五在后勤部。已为您提交2025年9月24日（下周三）的请假申请。'